# Importing the necessary libraries


In [142]:
import pandas as pd
import plotly
import plotly.express as ex
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Importing CSV Dataset

In [116]:
df = pd.read_csv('seaice.csv',names = ["Year","Month","Day","Extent","Missing","Hemisphere"],sep =';')
df.head()

,Year,Month,Day,Extent,Missing,Hemisphere
0,1978,10,26,10.231,0.0,north
1,1978,10,28,10.420,0.0,north
2,1978,10,30,10.557,0.0,north
3,1978,11,1,10.670,0.0,north
4,1978,11,3,10.777,0.0,north


In [117]:
df['Date'] = pd.to_datetime(df[['Year','Month','Day']])
df.index = df['Date'].values
df.head()

,Year,Month,Day,Extent,Missing,Hemisphere,Date
1978-10-26,1978,10,26,10.231,0.0,north,1978-10-26
1978-10-28,1978,10,28,10.420,0.0,north,1978-10-28
1978-10-30,1978,10,30,10.557,0.0,north,1978-10-30
1978-11-01,1978,11,1,10.670,0.0,north,1978-11-01
1978-11-03,1978,11,3,10.777,0.0,north,1978-11-03


## Comparing Northern and Southern hemisphere

In [118]:
north = df[df['Hemisphere'] == 'north']
south = df[df['Hemisphere'] == 'south']

#Using plotly
fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Ice extent (unit : 1O^6 squarred km)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=north.index,
                y=north['Extent'],
                name="Northern Hemisphere",
                line_color='red',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=south.index,
                y=south['Extent'],
                name="Southern Hemisphere",
                line_color='blue',
                opacity=0.8))

fig.update_layout(title_text = "Evolution of the ice extent between 1978 and 2019", title_x=0.5, \
                  title_font_size = 22)
fig.show()

We can see that there are obvious yearly fluctuations, with independent minima and maxima reached at the exact opposite period of the year in the northern hemisphere and in the southern hemisphere. These fluctuations are not really surprising as the ice extent is directly linked to the temperature and the temperature is linked to the season. To point out global warming let's focus on the evolution of the mean of these temperature.

In [123]:
northyear = north.groupby(['Year'])['Extent'].mean()
southyear = south.groupby(['Year'])['Extent'].mean()

#Using plotly
fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Ice extent (unit : 1O^6 squarred km)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2019)],
                y=northyear,
                name="Northern Hemisphere",
                line_color='red',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2019)],
                y=southyear,
                name="Southern Hemisphere",
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2019)],
                y=pd.Series.div(pd.Series.add(northyear,southyear),2) ,
                name="Global",
                line_color='green',
                mode = 'lines+markers',
                opacity=0.8))

fig.update_layout(title_text = "Evolution of the yearly average ice extent between 1978 and 2019", title_x=0.5, \
                  title_font_size = 22)
fig.show()


We can see that the yearly average describe a lot of fluctuations. However, the global trend in the northern hemisphere is clear, sea-ice is decreasing. In the Southern hemisphere the results are not exactly the same, until 2014, the sea-ice extent was fluctuating but maintening over years, but since 2014 there has been a huge decrease. The overall results over the past 36 years is clear for both hemispheres, the ice melting is really happenning.

## Predictions

In [147]:
#model : Linear regression

X = pd.DataFrame(north['Year'].unique())
X_pred = pd.DataFrame([i for i in range (1978, 2030)])
Y_north = northyear
Y_south = southyear

model = LinearRegression()

#Northern Hemisphere
model.fit(X,Y_north)
pred_north = model.predict(X_pred)

#Southern Hemisphere
model.fit(X,Y_south)
pred_south = model.predict(X_pred)

In [148]:
#Using plotly
fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Ice extent (unit : 1O^6 squarred km)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2019)],
                y=northyear,
                name="Northern Hemisphere",
                line_color='red',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2019)],
                y=southyear,
                name="Southern Hemisphere",
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2030)],
                y=pred_north ,
                name="Prediction Northern Hemisphere",
                line_color='red',
                mode = 'lines',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2030)],
                y=pred_south ,
                name="Prediction Southern Hemisphere",
                line_color='blue',
                mode = 'lines',
                opacity=0.8))

fig.update_layout(title_text = "Evolution and prediction of the yearly average ice extent between 1978 and 2030", title_x=0.5, \
                  title_font_size = 22)
fig.show()


Regarding this first prediction, the results are alarming in both hemispheres.

In [153]:
#Smoother with polynomial features
poly_reg = PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(X)
X_pred_poly = poly_reg.fit_transform(X_pred)

#North Hemisphere
poly_reg.fit(X_poly, Y_north)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, Y_north)
pred_poly_north = lin_reg.predict(X_pred_poly)

#South Hemisphere
poly_reg.fit(X_poly, Y_south)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, Y_south)
pred_poly_south = lin_reg.predict(X_pred_poly)

In [155]:
#Using plotly
fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Ice extent (unit : 1O^6 squarred km)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2019)],
                y=northyear,
                name="Northern Hemisphere",
                line_color='red',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2019)],
                y=southyear,
                name="Southern Hemisphere",
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2030)],
                y=pred_poly_north ,
                name="Prediction Northern Hemisphere",
                line_color='red',
                mode = 'lines',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1978,2030)],
                y=pred_poly_south ,
                name="Prediction Southern Hemisphere",
                line_color='blue',
                mode = 'lines',
                opacity=0.8))

fig.update_layout(title_text = "Evolution and prediction of the yearly average ice extent between 1978 and 2030", title_x=0.5, \
                  title_font_size = 22)
fig.show()

REgarding this new prediction, the recent and impressive decrease in the southern hemisphere affect a lot the alarming prediction in this area. Regarding the northern hemisphere, the sea ice extent seems to be likely to establish a new balance. These results are far from fitting exactly the reality, we should try other model that are likely to give better results. Maybe trying model that are taking into accoung CO2 emission as it migh be easier to predict the future CO2 emission from human activity and then from that prediction we could try to predict the future sea ice extent.